In [21]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from fastFM import als, mcmc, sgd
# from pyfm import pylibfm

import optuna

from common import EP
from dfdb import DFDB

import types
import copy

In [22]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [23]:
pd.set_option('display.max_colwidth', -1)

In [24]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [25]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [26]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [27]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [28]:
unique_columns = ['5000variance_median_',
 "number_peaks{'n': 5}",
 'spkt_welch_density__coeff_3',
 '5000quantile99median_',
 'q05_roll_std_1000',
 '5000skewness_max_',
 '5000form_factor_quantile75',
 'abs_q05_6',
 '5000smoothness_std_',
 'abs_q75_6',
 'q05_5',
 'abs_q75_2',
 'abs_q95_6',
 '5000smoothness_median_',
 'median__roll_std',
 'abs_q75_7',
 '5000smoothness_quantile25',
 'q75_roll_std_10',
 '5000median_variance_',
 'abs_q25_5',
 "number_peaks{'n': 1}",
 '5000smoothness_quantile05',
 'MA_1000MA_std_mean_7',
 'iqr_6',
 '5000std_median_',
 '3th_peak_freq',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'abs_q01_5',
 'spkt_welch_density__coeff_4',
 '5000variance_quantile25',
 'q01_2',
 '5000crest_factor_quantile75',
 '5000std_quantile05',
 'q01_roll_std_10',
 'iqr_3',
 'abs_q01_3',
 '5000std_quantile25',
 'spkt_welch_densitycoeff_2',
 'q01_roll_std_1000',
 "autocorrelation{'lag': 5}",
 "value_count{'value': 1}",
 "number_peaks{'n': 3}",
 'abs_q95_2',
 "number_peaks{'n': 10}",
 '5000smoothness_mean_',
 '5000smoothness_entropy_',
 'q05_roll_std_100',
 'q01_roll_std_100',
 'abs_q01_4',
 "quantile{'q': 0.9}",
 'q05_roll_std_10',
 'FFT_Mag_75q0',
 'q25_roll_std_100']

In [29]:
common_columns = ['q05_roll_std_1000',
 'abs_q75_6',
 'abs_q25_5',
 'iqr_6',
 "number_peaks{'n': 10}",
 'q05_roll_std_100',
 'abs_q01_4',
 'q25_roll_std_100']

In [30]:
common_columns75 = ['spkt_welch_density__coeff_3',
 'q05_roll_std_1000',
 'abs_q75_6',
 'q05_5',
 'median__roll_std',
 'abs_q75_7',
 'abs_q25_5',
 '5000smoothness_quantile05',
 'iqr_6',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'spkt_welch_densitycoeff_2',
 "number_peaks{'n': 10}",
 'q05_roll_std_100',
 'abs_q01_4',
 'q05_roll_std_10',
 'q25_roll_std_100']

In [31]:
# mytrial = []
db = DFDB('../trial/knn.pkl', auto_commit=False)

In [38]:
param = {'algorithm': {'cls': 'KNeighborsRegressor',
  'fit': {},
  'init': {'n_neighbors': 500,
   'weights': 'uniform',
   'algorithm': 'ball_tree',
   'leaf_size': 30,
   'p': 2,
   'metric': 'minkowski'}},
 'columns':common_columns ,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler'}}

In [39]:
# run one try
mytrial = []
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial, remark='try common_columns')
db.insert(mytrial[0])
df_trial = db.select()

In [40]:
df_trial[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
0,2019-05-19 03:03:20.036697,16,2.024692,0.002800,2.075379,0.013900,0.050687,try common_columns75
1,2019-05-19 03:04:25.860199,8,2.044007,0.002525,2.081530,0.012794,0.037523,try common_columns


In [41]:
mytrial = []
#  tune hypterparameters
def objective(trial):

    n_neighbors = trial.suggest_int('n_neighbors', 100, 1000)
        
    args={'algorithm': {'cls': 'KNeighborsRegressor',
  'fit': {},
  'init': {'n_neighbors': n_neighbors,
   'weights': 'uniform',
   'algorithm': 'ball_tree',
   'leaf_size': 30,
   'p': 2,
   'metric': 'minkowski'}},
 'columns':common_columns ,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler'}}
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 0')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=100)

[I 2019-05-19 03:06:15,308] Finished trial#0 resulted in value: 0.07318632536974794. Current best value is 0.07318632536974794 with parameters: {'n_neighbors': 569}.
[I 2019-05-19 03:07:04,573] Finished trial#1 resulted in value: 0.061117887181583924. Current best value is 0.061117887181583924 with parameters: {'n_neighbors': 759}.
[I 2019-05-19 03:07:58,483] Finished trial#2 resulted in value: 0.056168214996267216. Current best value is 0.056168214996267216 with parameters: {'n_neighbors': 884}.
[I 2019-05-19 03:08:37,269] Finished trial#3 resulted in value: 0.09430557063087121. Current best value is 0.056168214996267216 with parameters: {'n_neighbors': 884}.
[I 2019-05-19 03:09:45,239] Finished trial#4 resulted in value: 0.05406133522384894. Current best value is 0.05406133522384894 with parameters: {'n_neighbors': 937}.
[I 2019-05-19 03:10:40,348] Finished trial#5 resulted in value: 0.06259075830185049. Current best value is 0.05406133522384894 with parameters: {'n_neighbors': 937}.

[I 2019-05-19 03:43:08,334] Finished trial#49 resulted in value: 0.09627099135150795. Current best value is 0.05199563426970925 with parameters: {'n_neighbors': 997}.
[I 2019-05-19 03:44:06,478] Finished trial#50 resulted in value: 0.054785515805889536. Current best value is 0.05199563426970925 with parameters: {'n_neighbors': 997}.
[I 2019-05-19 03:44:35,615] Finished trial#51 resulted in value: 0.10167386695565821. Current best value is 0.05199563426970925 with parameters: {'n_neighbors': 997}.
[I 2019-05-19 03:45:25,328] Finished trial#52 resulted in value: 0.05851229990750937. Current best value is 0.05199563426970925 with parameters: {'n_neighbors': 997}.
[I 2019-05-19 03:46:16,813] Finished trial#53 resulted in value: 0.05753274955387322. Current best value is 0.05199563426970925 with parameters: {'n_neighbors': 997}.
[I 2019-05-19 03:46:52,471] Finished trial#54 resulted in value: 0.08092855058538002. Current best value is 0.05199563426970925 with parameters: {'n_neighbors': 997

[I 2019-05-19 04:24:17,139] Finished trial#98 resulted in value: 0.12835303022939196. Current best value is 0.05187819023918285 with parameters: {'n_neighbors': 999}.
[I 2019-05-19 04:24:52,628] Finished trial#99 resulted in value: 0.07322823898739679. Current best value is 0.05187819023918285 with parameters: {'n_neighbors': 999}.


In [42]:
for trial_i in mytrial:
    db.insert(trial_i)

In [43]:
db.commit()